## 1. Libraries & EDA

In [ ]:
# data science
import numpy as np
import pandas as pd

# deep learning & model training
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras


# hyperparam tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error

# Model selection
import mlflow
from mlflow.models import infer_signature

# path
import os


In [2]:
# Import sample data - wine quality
url = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/whitewines.csv"
raw_df = pd.read_csv(url,
                     sep = ',')

raw_df.head() # Quality is the target output

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
# Check for missing values
raw_df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [4]:
# Look at summary of measures
raw_df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


### 2. Data Preparation

In [5]:
# Preparation of training data
train, test = train_test_split(raw_df, test_size = 0.25, random_state = 42)


In [6]:
# Extracting the array of both training and testing dataset
train_x = train.drop(['quality'], axis = 1).values
train_y = train[['quality']].values.ravel() # ravel() flattens the data of n-dimension array

# Testing dataset - so the data will not see this dataset
test_x = test.drop(['quality'], axis = 1).values
test_y = test[['quality']].values.ravel()

In [7]:
# Prepare training & validation data for the model training
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size = 0.2, random_state = 42)

# Inferring model signature from both input & output
signature = infer_signature(train_x, train_y)

### 3. Training our ANN model


In [8]:
train_x.shape[1]

11

In [9]:
# Function to run a full function

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    # Performing column-wise normalization
    mean = np.mean(train_x, axis = 0)
    var = np.var(train_x, axis = 0)

    # Define model architecture
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean = mean, variance = var),
            keras.layers.Dense(64, activation = 'relu'),
            keras.layers.Dense(1)
        ]
    )

    # Compiling the model
    model.compile(
        optimizer = keras.optimizers.SGD(
            learning_rate = params['lr'],
            momentum = params['momentum']
        ),
        loss = "mean_squared_error",
        metrics = [keras.metrics.RootMeanSquaredError()]
    )

    # Training the model & track them with mlflow
    with mlflow.start_run(nested = True):
        model.fit(train_x, train_y, validation_data = (valid_x, valid_y),
                  epochs = epochs,
                  batch_size = 64
                  )
        
        # Model evaluation
        eval_result = model.evaluate(valid_x, valid_y, batch_size = 64)

        eval_rmse = eval_result[1]

        # Log the params & reuslts
        mlflow.log_params(params)
        mlflow.log_metric("evaluation_rmse", eval_rmse)

        # Log the model & return the status
        mlflow.tensorflow.log_model(model, "model", signature = signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [10]:
## hyperopt objective function for MLflow to track
def objective(params):
    # For MLflow to track the parameters and results for each run
    result = train_model(
        params,
        epochs = 3,
        train_x = train_x,
        train_y = train_y,
        valid_x = valid_x,
        valid_y = valid_y,
        test_x = test_x,
        test_y = test_y
    )
    return result


In [11]:
# Define the param-variables 
space = {
    "lr": hp.loguniform('lr', np.log(1e-5), np.log(1e-1)),
    'momentum': hp.uniform('momentum', 0.0, 1.0)
}

In [12]:
mlflow.set_experiment("../white-wine-quality")

with mlflow.start_run():
    # Conduct hyperparam search using hyperopt
    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo = tpe.suggest,
        max_evals = 4,
        trials = trials
    )

    # Fetch the details 
    best_run = sorted(trials.results, key = lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric('eval_rmse', best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'], name = 'model', signature = signature)

    # Print best params & loss
    print(f"Best parameters: {best}")
    print(f"Best evaluated RMSE: {best_run['loss']}")

2026/01/26 15:20:38 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/01/26 15:20:38 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/01/26 15:20:38 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/01/26 15:20:38 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/01/26 15:20:38 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/01/26 15:20:38 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/01/26 15:20:38 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/26 15:20:38 INFO mlflow.store.db.utils: Updating database tables
2026/01/26 15:20:38 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/26 15:20:38 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/26 15:20:38 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/26 15:20:38 INFO alembic.runtime

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

2026-01-26 15:20:38.302674: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2026-01-26 15:20:38.302847: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2026-01-26 15:20:38.302854: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2026-01-26 15:20:38.303295: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-26 15:20:38.303316: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/3                                            

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

2026-01-26 15:20:38.570680: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 10s 230ms/step - loss: 35.8701 - root_mean_squared_error: 5.9892
14/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.9983 - root_mean_squared_error: 5.9998   
28/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.9635 - root_mean_squared_error: 5.9969
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.8806 - root_mean_squared_error: 5.9900
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 35.5732 - root_mean_squared_error: 5.9643 - val_loss: 35.4813 - val_root_mean_squared_error: 5.9566

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 37.7338 - root_mean_squared_error: 6.1428
15/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 36.0644 - root_mean_squared_error: 6.0052 
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.6452 - root_mean_squared_error: 5.9701
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.5285 - ro

2026/01/26 15:20:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 10s 237ms/step - loss: 37.3207 - root_mean_squared_error: 6.1091
14/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 16.4412 - root_mean_squared_error: 3.9065   
28/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10.7078 - root_mean_squared_error: 3.0644
42/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.2039 - root_mean_squared_error: 2.6382 
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.5944 - root_mean_squared_error: 1.6107 - val_loss: 0.6148 - val_root_mean_squared_error: 0.7841

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.6286 - root_mean_squared_error: 0.7929
15/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6727 - root_mean_squared_error: 0.8200 
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6600 - root_mean_squared_error: 0.8123
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6482 - root_mea

2026/01/26 15:20:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:21<00:00,  5.40s/trial, best loss: 0.8028776049613953]
Best parameters: {'lr': 0.04181875089060582, 'momentum': 0.22563247666543063}
Best evaluated RMSE: 0.8028776049613953


### 4. Output data for inferencing 'best' model

In [13]:
# Create 'data' directory if it doesn't exist
os.makedirs("data", exist_ok = True)

# Save to csv for usage later
test.to_csv("data/test.csv", index = False)
